# Laboratorio de Introducción al Procesamiento de Lenguaje Natural (2024)



---


En este laboratorio vamos a poner en práctica algunos de los conceptos vistos en el curso, como la *mínima distancia de edición* para medir la cercanía entre dos tiras, los *embeddings* como representación semántica de los textos, los *modelos de n-gramas* para generar texto y un problema de clasificación de tres clases.

Los ejercicios que tienen que resolver están señalizados por números del 0️⃣al 7️⃣. Los resultados y conclusiones que surjan de esos ejercicios los van a comunicar en un informe (con un **máximo de 6 páginas**). En la sección "¿Qué se espera del informe?" de la letra del laboratorio van a encontrar una guía para escribirlo.



---



Antes que nada, vamos a instalar e importar NLTK y descargar ["Cuentos de Amor de Locura y de Muerte"](https://es.wikipedia.org/wiki/Cuentos_de_amor_de_locura_y_de_muerte) de Horacio Quiroga del [repositorio Gutemberg](https://www.gutenberg.org/ebooks/13507). Esto ya está implementado, y el libro va a quedar guardado en la variable `quiroga_raw`.

In [3]:
%%capture
import re
import nltk
nltk.download('punkt')

!wget https://www.gutenberg.org/cache/epub/13507/pg13507.txt
with open("pg13507.txt","r") as f:
    quiroga_raw = f.read()


## 0️⃣Limpieza básica del corpus

Si examinamos el string cargado en `quiroga_raw`, vemos que al inicio y al final hay texto en inglés, además de que algunos saltos de línea ocurren a mitad de las oraciones.

Como primer paso, diseñe expesiones regulares que limpien `quiroga_raw` para que:


* quede solamente el texto que está entre "#Cuentos de Amor de Locura y de Muerte#" y "FIN\n"
* saquen aquellos saltos de línea que tengan a su derecha una palabra que comience con letra minúscula  

El texto limpio resultante deberá quedar guardado en la variable `quiroga`. Más adelante van a poder probar con otros preprocesamientos complementarios a esta limpieza básica.



In [4]:
#En esta celda les dejamos esta asignación para que el notebook sea funcional.
#Implementen en esta celda la limpieza correspondiente.

# 1. Extracción del texto
print("Extrayendo texto...")
quiroga_text_extract_pattern = r"#Cuentos de Amor de Locura y de Muerte#(.*)FIN\n"  # Expresion regular
match = re.search(quiroga_text_extract_pattern, quiroga_raw, re.DOTALL)
if match:
    quiroga_text = match.group(1).strip()  # Remuevo espacios en blanco
    print(f"Texto original: {len(quiroga_raw)} caracteres. \nTexto extraído: {len(quiroga_text)} caracteres.")
else:
    print("Algo anda mal!")

# 2. Limpieza saltos de línea
print("\n\nLimpiando saltos de línea en sentencias...")
breakline_lowercase_pattern = r"\n([a-z])"
quiroga = re.sub(breakline_lowercase_pattern, r" \1", quiroga_text)  # Reemplazo \n por un ' '
print(f"Texto original: {len(quiroga_text)} caracteres. \nTexto extraído: {len(quiroga)} caracteres.")

print("\n\n")
print("Texto Limpio\n")
print(quiroga[:1000])

Extrayendo texto...
Texto original: 312980 caracteres. 
Texto extraído: 293434 caracteres.


Limpiando saltos de línea en sentencias...
Texto original: 293434 caracteres. 
Texto extraído: 293434 caracteres.



Texto Limpio

HORACIO QUIROGA

1917




#INDICE#


Una estación de amor
Los ojos sombríos
El solitario
La muerte de Isolda
El infierno artificial
La gallina degollada
Los buques suicidantes
El almohadón de pluma
El perro rabioso
A la deriva
La insolación
El alambre de púa
Los Mensú
Yaguaí
Los pescadores de vigas
La miel silvestre
Nuestro primer cigarro
La meningitis y su sombra









#UNA ESTACION DE AMOR#




#Primavera#


Era el martes de carnaval. Nébel acababa de entrar en el corso, ya al oscurecer, y mientras deshacía un paquete de serpentinas, miró al carruaje de delante. Extrañado de una cara que no había visto la tarde anterior, preguntó a sus compañeros:


--¿Quién es? No parece fea.

--¡Un demonio! Es lindísima. Creo que sobrina, o cosa así, del doctor
Arrizabalaga. 

## Primera parte: Modelos de lenguaje de n-gramas

Ahora que tenemos el texto más limpio, vamos a entrenar modelos de lenguaje de n-gramas sobre él. Para eso, les vamos a dar definidas e implementadas las funciones:
- `train_language_model`, que dado un natural `n` y un `corpus`, entrena un modelo de lenguaje basado en n-gramas. El parámetro `n` tiene el valor 4 por defecto.
- `language_model_inference`, que dado un modelo de lenguaje ya entrenado y una `prompt`, da como salida tantos tokens sucesivos como indique la variable `length`. Este largo tiene por defecto el valor 25, y la semilla `seed` tiene por defecto el valor 2024.

In [5]:
def train_language_model (corpus, n = 4):
  padded_tokens = [list(nltk.lm.preprocessing.pad_both_ends(nltk.tokenize.word_tokenize(corpus, language='spanish'), n=n))]

  train, vocab = nltk.lm.preprocessing.padded_everygram_pipeline(n, padded_tokens)

  language_model = nltk.lm.MLE(n)
  language_model.fit(train,vocab)

  return language_model

def language_model_inference (language_model, prompt, length = 25, seed = 2024):
  tokens = language_model.generate(length, text_seed=nltk.tokenize.word_tokenize(prompt), random_seed=seed)
  return tokens


### 1️⃣Entrenamiento e inferencia

Ahora que ya están definidas las funciones de entrenamiento e inferencia de los modelos de lenguaje, entrenen diferentes modelos variando:
- el parámetro `n`, por ejemplo con `n=3`, `n=2` y el valor por defecto, `n=4`
- el parámetro `corpus`, que se consigue al probar con diferentes pipelines de preprocesamieto. Pueden intentar pasar todo a minúsculas, sustituir los saltos de línea por espacios en blanco y todo lo que tengan ganas de probar. También pueden incluir el experimento de no preprocesar; es decir, que el pipeline de preprocesamiento sea la identidad.

Luego, con cada uno de esos modelos, generen texto pasándole como `prompt`,  "las vacas", "el cielo", y una opción más que encuentren interesante. Aunque no es necesario, pueden probar otros valores de `length` y `seed`, más allá de los que están por defecto.

💡 Les recomendamos que cada pipeline de preprocesamiento sea una función independiente. De esa manera, se facilita la reproducibilidad de los experimentos y su comparación, e.g. `n=3` preprocesando con `pipeline1()`, `n=3` con `pipeline2()`, `n=2` con `pipeline1()` y `n=2` con `pipeline2()`.



In [ ]:
#Completar con los diferentes experimentos, llamando a las funciones train_language_model y language_model_inference

"""
Experimentos a realizar:

1. Variar n en [4, 3, 2]
2. Jugar con pre-procesamientos.

Pre-procesamientos:
1. Nada
2. lowercase
3. replace(\n, " ")
"""

def pipeline_1():
  """
  Este pipeline entrena modelo con n=4 y sin pre-procesamiento al corpus (más allá del inicial)
  """
  print(f"pipeline_1 -- Parámetros: n={4}, pre-process=None")
  language_model = train_language_model(corpus=quiroga, n=4)
  return language_model

def pipeline_2():
  """
  Este pipeline entrena modelo con n=3 y sin pre-procesamiento al corpus (más allá del inicial)
  """
  print(f"pipeline_2 -- Parámetros: n={3}, pre-process=None")
  language_model = train_language_model(corpus=quiroga, n=3)
  return language_model

def pipeline_3():
  """
  Este pipeline entrena modelo con n=3 y sin pre-procesamiento al corpus (más allá del inicial)
  """
  print(f"pipeline_3 -- Parámetros: n={2}, pre-process=None")
  language_model = train_language_model(corpus=quiroga, n=2)
  return language_model

def pipeline_4():
  """
  Este pipeline entrena modelo con n=4 y lowercase al corpus
  """
  print(f"pipeline_4 -- Parámetros: n={4}, pre-process=lowercase()")

  # Pre-procesamiento
  pre_process_corpus = quiroga.lower()

  # Entrenamiento
  language_model = train_language_model(corpus=pre_process_corpus, n=4)
  return language_model

def pipeline_5():
  """
  Este pipeline entrena modelo con n=4 y replace(\n, " ").
  """
  print(f"pipeline_5 -- Parámetros: n={4}, pre-process=replace(\n, ' ')")

  # Pre-procesamiento
  pre_process_corpus = quiroga.replace("\n", " ")

  # Entrenamiento
  language_model = train_language_model(corpus=pre_process_corpus, n=4)
  return language_model

# Generamos los language models
models = [pipeline_1(), pipeline_2(), pipeline_3(), pipeline_4(), pipeline_5()]

# Generamos textos
texts = ["las vacas", "el cielo", "Esteban Podeley"]
for text in texts:

  print(f"\nGenerando para text='{text}'")
  text_results = []
  for i, model in enumerate(models):
    output_tokens = language_model_inference(language_model=model, prompt=text)
    model_name = f"pipeline_{i+1}"
    print(f"model='{model_name}' output={output_tokens}")

pipeline_1 -- Parámetros: n=4, pre-process=None
pipeline_2 -- Parámetros: n=3, pre-process=None
pipeline_3 -- Parámetros: n=2, pre-process=None
pipeline_4 -- Parámetros: n=4, pre-process=lowercase()
pipeline_5 -- Parámetros: n=4, pre-process=replace(
, ' ')

Generando para text='las vacas'
model='pipeline_1' output=['dormitaban', 'al', 'sol', 'ya', 'caliente', ',', 'rumiando', '.', 'Pero', 'cuando', 'está', 'conmigo', ',', 'entonces', 'no', 'aparta', 'los', 'ojos', 'de', 'mi', 'mujer', 'y', 'yo', ',', 'con']
model='pipeline_2' output=['dormitaban', 'al', 'sol', 'ya', 'caliente', ',', 'rumiando', '.', 'Pero', 'en', 'el', 'patio', 'y', 'Alfonso', 'la', 'llamó', 'en', 'silencio', '.', 'Pasábanse', 'horas', 'sin', 'oir', 'el', 'angustioso']
model='pipeline_3' output=['dormitaban', 'al', 'fin', 'se', 'había', 'reforzado', 'su', 'corazón', 'siempre', 'en', 'la', 'pipa', 'y', 'el', 'perro', 'había', 'retirado', '.', 'A', 'media', 'hora', 'en', 'tierra', ',', 'del']
model='pipeline_4' output=[

OBSERVACION (@eviotti): El pipeline_4 parece generar sentencias parecidas a las que se encuentran en el texto. N=4 parece generar resultados más pareceidos en el uso del idiona.

### 2️⃣ Mínima distancia de edición

Los modelos de lenguaje que entrenaron en la parte anterior aprendieron la distribución estadística a través de secuencias de palabras en el libro de Quiroga. Sin embargo, las inferencias que consiguieron ¿estarán idénticas en el corpus de entrenamiento?

Usando la función `edit_distance` de NLTK encuentren, para cada tira generada en la parte anterior, las 3 oraciones del libro original que tienen la menor distancia de edición asociada (es decir, el top 3). Las oraciones del libro están guardadas en la variable `quiroga_sentences`.

In [ ]:
quiroga_sentences = nltk.sent_tokenize(quiroga, language='spanish')

In [ ]:
print(f'La distancia entre "arbolada" y "escapada" es {nltk.edit_distance("arbolada", "escapada")}, pero entre "arbolada" y "arbolado" es {nltk.edit_distance("arbolada", "arbolado")}.')

La distancia entre "arbolada" y "escapada" es 5, pero entre "arbolada" y "arbolado" es 1.


In [ ]:
import numpy as np

def get_knn(text: str, k: int = 3, sentences: list[str] = []) -> list[str]:
  # Obtengo distancias
  distances = [nltk.edit_distance(text, sentence) for sentence in sentences]
  # Obtengo indices de los K mas cercanos
  closest_indices = np.argsort(distances)[:k]
  closest_sentences = [sentences[i] for i in closest_indices]

  return closest_sentences

# Para guardar los resultados
predictions = []

for text in texts:

  print(f"\n\nGenerando para text='{text}'")
  for i, model in enumerate(models):
    # Genero texto
    model_name = f"pipeline_{i+1}"
    output_tokens = language_model_inference(language_model=model, prompt=text)
    output_text = " ".join(output_tokens)

    print(f"model='{model_name}' output='{output_text}'")

    # Obtengo top_k vectores mas cercanos
    nearest_k = get_knn(text=output_text, k=3, sentences=quiroga_sentences)
    print(f"neighbors={nearest_k}")

    for idx, neighbor_vec in enumerate(nearest_k):
      # Guardo resultados
      predictions.append({
                  "prompt": text,
                  "model_name": model_name,
                  "output_text": output_text,
                  "idx": idx+1,
                  "neighborh": neighbor_vec
              })



Generando para text='las vacas'
model='pipeline_1' output='dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con'
neighbors=['Pero cuando está conmigo, entonces no aparta los ojos de ellos.', 'Arrizabalaga y la señora se reían, volviéndose a menudo, y la joven no apartaba casi sus ojos de Nébel.', 'El caballo, por mayor intimidad de trato, es sensiblemente más afecto al hombre que la vaca.']
model='pipeline_2' output='dormitaban al sol ya caliente , rumiando . Pero en el patio y Alfonso la llamó en silencio . Pasábanse horas sin oir el angustioso'
neighbors=['Celia, mi tía mayor, que había concluído de dormir la siesta, cruzó el patio y Alfonso la llamó en silencio con la mano.', 'Al bajar el sol volvieron, pero Berta quiso saludar un momento a sus vecinas de enfrente.', 'Volvió a su cobertizo, y en el camino sintió un ligero cosquilleo en la espalda.']
model='pipeline_3' output='dormitaban al fin se había reforzado su

In [ ]:
import pandas as pd

# Creo DataFrame
df = pd.DataFrame(predictions, columns=["prompt", "model_name", "output_text", "idx", "neighborh"])

In [ ]:
df

,prompt,model_name,output_text,idx,neighborh
0,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",1,"Pero cuando está conmigo, entonces no aparta l..."
1,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",2,"Arrizabalaga y la señora se reían, volviéndose..."
2,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",3,"El caballo, por mayor intimidad de trato, es s..."
3,las vacas,pipeline_2,"dormitaban al sol ya caliente , rumiando . Per...",1,"Celia, mi tía mayor, que había concluído de do..."
4,las vacas,pipeline_2,"dormitaban al sol ya caliente , rumiando . Per...",2,"Al bajar el sol volvieron, pero Berta quiso sa..."
5,las vacas,pipeline_2,"dormitaban al sol ya caliente , rumiando . Per...",3,"Volvió a su cobertizo, y en el camino sintió u..."
6,las vacas,pipeline_3,dormitaban al fin se había reforzado su corazó...,1,"Como las fieras amaestradas, los perros conoce..."
7,las vacas,pipeline_3,dormitaban al fin se había reforzado su corazó...,2,Pasaban casi todo el día sentados frente al ce...
8,las vacas,pipeline_3,dormitaban al fin se había reforzado su corazó...,3,"Por lo demás, se alternaban con su hija para i..."
9,las vacas,pipeline_4,"estaban inmóviles , mirando fijamente el verde...",1,"Las vacas estaban inmóviles, mirando fijamente..."


### 3️⃣ Modelo neuronal para capturar la semántica

Ahora repitan el mismo procedimiento de la parte anterior pero usando el [modelo neuronal E5](https://huggingface.co/intfloat/multilingual-e5-large), basado en la arquitectura BERT, que intenta representar el significado de un texto en un vector de largo fijo.

Para hallar los tres vectores más cercanos a un vector dado, utilicen el algoritmo [Nearest Neighbors implementado en Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html).

💡Van a tener que codificar (con vectores) tanto lo generado por el modelo de n-gramas como todas las oraciones guardadas en `quiroga_sentences`.

💡Asegúrense de estar usando una GPU en el entorno de ejecución de Google Colab, para poder acelerar el procesamiento con E5.

In [ ]:
#Completar con la instalación de las bibliotecas necesarias,
#el código que halla la representación vectorial de los textos
#y el algoritmo para hallar el top 3 con Nearest Neighbors.

import torch
from torch.utils.data import DataLoader
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def generate_vector_from_text(batch, model, tokenizer, device):
  # Tokenize the batch
  batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)

  with torch.no_grad():  # Disable gradient calculation
      outputs = model(**batch_dict)
      embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
      embeddings_cpu = embeddings.cpu()  # Move embeddings to CPU to save GPU memory

  # Clear memory after each batch
  del outputs, embeddings, batch_dict
  torch.cuda.empty_cache() if torch.cuda.is_available() else None

  return embeddings_cpu

def generate_vectors_from_texts(texts, model, tokenizer, device):
  texts_embeddings = []

  # Defino data Loader para cargar eficientemente los vectores
  batch_size = 16  # Adjust the batch size based on your memory availability
  data_loader = DataLoader(texts, batch_size=batch_size, shuffle=False)

  for batch in data_loader:
    batch_embeddings = generate_vector_from_text(batch=batch, model=model, tokenizer=tokenizer, device=device)
    texts_embeddings.append(batch_embeddings)

  # Concatenate all embeddings after processing
  texts_embeddings = torch.cat(texts_embeddings, dim=0)
  return texts_embeddings

In [ ]:
# En esta celda generamos los vectores para todas las sentencias de
# quiroga y para los textos de prueba

# Device GPU si hay
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Init tokenizer y Modelo
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
model.to(device)

# 1. Texts
input_texts =  ["las vacas", "el cielo", "Esteban Podeley"]
texts_embeddings = generate_vectors_from_texts(texts=input_texts, model=model, tokenizer=tokenizer, device=device)
print(texts_embeddings.shape)

# 2. Quiroga Sentences
quiroga_embeddings = generate_vectors_from_texts(texts=quiroga_sentences, model=model, tokenizer=tokenizer, device=device)
print(quiroga_embeddings.shape)


torch.Size([3, 1024])
torch.Size([3540, 1024])


In [ ]:
# En esta celda calculamos los vecinos más cercanos
from sklearn.neighbors import NearestNeighbors

# 3. Hallo top 3 con Nearest Neighbors.
neigh = NearestNeighbors(n_neighbors=3, radius=0.4)
neigh.fit(quiroga_embeddings)

predictions = []  # Para guardar las predicciones

# Corro knn en batch para todos los texts
distances, indices = neigh.kneighbors(embeddings.cpu().detach().numpy(), 3, return_distance=True)
print(f"Distances: {distances.shape}")
print(f"Indices: {indices.shape}")

for i, (text_distances, text_neighboors) in enumerate(zip(distances, indices)):
  for j, (distance, n_idx) in enumerate(zip(text_distances, text_neighboors)):
    neighboor = quiroga_embeddings[i]
    predictions.append({"prompt": input_texts[i], "idx": j+1, "neighborh": quiroga_sentences[n_idx]})

Distances: (3, 3)
Indices: (3, 3)


In [ ]:
import pandas as pd
# Creo DataFrame
df = pd.DataFrame(predictions, columns=["prompt", "idx", "neighborh"])
df

,prompt,idx,neighborh
0,las vacas,1,Pero las vacas lo habían oído.
1,las vacas,2,¡Vaca tiene culpa!
2,las vacas,3,"Las vacas, mientras tanto, se animaban unas a ..."
3,el cielo,1,es mío.
4,el cielo,2,¡Para mí!
5,el cielo,3,--El pasó ayer.
6,Esteban Podeley,1,"Podeley jamás había dejado de cumplir nada, ún..."
7,Esteban Podeley,2,#LOS MENSÚ#\n\n\n\n\nCayetano Maidana y Esteba...
8,Esteban Podeley,3,"Podeley, libre hasta entonces, sintióse un día..."


## Segunda parte: Clasificación de textos por autor

En esta segunda parte del laboratorio vamos a usar los datos limpios de `quiroga` y todo lo practicado de manipulación de tiras para hacer experimentos de clasificación.


Con el fin de construir modelos que intenten identificar los autores de cada oración según su estilo de escritura, vamos a incorporar dos libros más, una [selección de obras](https://www.gutenberg.org/ebooks/53552) de Gustavo Adolfo Bécquer y [El Gaucho Martín Fierro](https://es.wikipedia.org/wiki/El_Gaucho_Mart%C3%ADn_Fierro) de José Hernández.

In [1]:
%%capture
!wget https://www.gutenberg.org/cache/epub/53552/pg53552.txt
with open("pg53552.txt","r") as f:
    becquer_raw = f.read()

!wget https://www.gutenberg.org/cache/epub/14765/pg14765.txt
with open("pg14765.txt","r") as f:
    martin_fierro_raw = f.read()

### 4️⃣ Limpieza básica del nuevo texto


Análogamente a lo hecho en el ejercicio 0️⃣, limpien `becquer_raw` para que en la variable `becquer` quede:


* solamente el texto que está entre "Junio de 1868." y "FIN\n"
* sin aquellos saltos de línea que tengan a su derecha una palabra que comience con letra minúscula
* y borrando todas las subtiras "[Ilustración]"
  

In [7]:
#En esta celda les dejamos esta asignación para que el notebook sea funcional.
#Implementen en esta celda la limpieza correspondiente.
#becquer = becquer_raw

# 1. Extracción del texto
print("Extrayendo texto...")
becquer_text_extract_pattern = r"Junio de 1868\.(.*)FIN\n"  # Expresion regular
match = re.search(becquer_text_extract_pattern, becquer_raw, re.DOTALL)
if match:
    becquer_text = match.group(1).strip()  # Remuevo espacios en blanco
    print(f"Texto original: {len(becquer_raw)} caracteres. \nTexto extraído: {len(becquer_text)} caracteres.")
else:
    print("Algo anda mal!")

# 2. Limpieza saltos de línea
print("\n\nLimpiando saltos de línea en sentencias...")
breakline_lowercase_pattern = r"\n([a-z])"
becquer = re.sub(breakline_lowercase_pattern, r" \1", becquer_text)  # Reemplazo \n por un ' '
print(f"Texto original: {len(becquer_text)} caracteres. \nTexto extraído: {len(becquer)} caracteres.")

# 3. Borro subturas ilustración
becquer = becquer.replace("[Ilustración]", " ")

print("\n\n")
print("Texto Limpio\n")
print(becquer[:1000])

Extrayendo texto...
Texto original: 573195 caracteres. 
Texto extraído: 527105 caracteres.


Limpiando saltos de línea en sentencias...
Texto original: 527105 caracteres. 
Texto extraído: 527105 caracteres.



Texto Limpio

 




LEYENDAS




 

MAESE PÉREZ EL ORGANISTA


En Sevilla, en el mismo atrio de Santa Inés, y mientras esperaba que comenzase la Misa del Gallo, oí esta tradición á una demandadera del convento.

Como era natural, después de oirla, aguardé impaciente que comenzara la ceremonia, ansioso de asistir á un prodigio.

Nada menos prodigioso, sin embargo, que el órgano de Santa Inés, ni nada más vulgar que los insulsos motetes que nos regaló su organista aquella noche.

Al salir de la Misa, no pude por menos de decirle á la demandadera con aire de burla:

--¿En qué consiste que el órgano de maese Pérez suena ahora tan mal?

--¡Toma!--me contestó la vieja,--en que ese no es el suyo.

--¿No es el suyo? ¿Pues qué ha sido de él?

--Se cayó á pedazos de puro viejo, hace una po

Hagan lo mismo con `martin_fierro_raw`, guardando en la variable `martin_fierro`:
* el texto que está entre "Buenos Aires, diciembre de 1872." y "End of Project"
* sin aquellos saltos de línea que tengan a su derecha una palabra que comience con letra minúscula
* y borrando todos los números de entre 1 y 3 cifas, e.g. 7, 12, 178   

⚠️ Aclaración: *El Gaucho Martín Fierro* está originalmente escrito como un poema, por lo que los saltos de línea no tienen el mismo fin que en la prosa. Sin embargo, eliminaremos esos saltos de línea seguidos de minúsculas como convención, a fin de facilitar el ejercicio y hacer más parecido el texto a los guardados en `quiroga` y `becquer`.

In [10]:
#En esta celda les dejamos esta asignación para que el notebook sea funcional.
#Implementen en esta celda la limpieza correspondiente.
#martin_fierro = martin_fierro_raw

# 1. Extracción del texto
print("Extrayendo texto...")
martin_fierro_text_extract_pattern = r"Buenos Aires, diciembre de 1872\.(.*)End of Project"  # Expresion regular
match = re.search(martin_fierro_text_extract_pattern, martin_fierro_raw, re.DOTALL)
if match:
    martin_fierro_text = match.group(1).strip()  # Remuevo espacios en blanco
    print(f"Texto original: {len(martin_fierro_raw)} caracteres. \nTexto extraído: {len(martin_fierro_text)} caracteres.")
else:
    print("Algo anda mal!")

# 2. Limpieza saltos de línea
print("\n\nLimpiando saltos de línea en sentencias...")
breakline_lowercase_pattern = r"\n([a-z])"
martin_fierro = re.sub(breakline_lowercase_pattern, r" \1", martin_fierro_text)  # Reemplazo \n por un ' '
print(f"Texto original: {len(martin_fierro_text)} caracteres. \nTexto extraído: {len(martin_fierro)} caracteres.")

# 3. Borro numeros de 1-3 cifras
martin_fierro = martin_fierro.replace("[Ilustración]", " ")
martin_fierro = re.sub(r'\b\d{1,3}\b', ' ', martin_fierro)

print("\n\n")
print("Texto Limpio\n")
print(martin_fierro[:1000])

Extrayendo texto...
Texto original: 85812 caracteres. 
Texto extraído: 62129 caracteres.


Limpiando saltos de línea en sentencias...
Texto original: 62129 caracteres. 
Texto extraído: 62129 caracteres.



Texto Limpio

El Gaucho Martín Fierro


I - Cantor y Gaucho.

 
Aquí me pongo a cantar
Al compás de la vigüela,
Que el hombre que lo desvela
Una pena estraordinaria
Como la ave solitaria
Con el cantar se consuela.

 
Pido a los Santos del Cielo
Que ayuden mi pensamiento;
Les pido en este momento
Que voy a cantar mi historia
Me refresquen la memoria
Y aclaren mi entendimiento.

 
Vengan Santos milagrosos,
Vengan todos en mi ayuda,
Que la lengua se me añuda
Y se me turba la vista;
Pido a Dios que me asista
En una ocasión tan ruda.

 
Yo he visto muchos cantores,
Con famas bien obtenidas,
Y que después de adquiridas
No las quieren sustentar
Parece que sin largar se cansaron en partidas

 
Mas ande otro criollo pasa
Martín Fierro ha de pasar; nada lo hace recular ni los fantasmas lo espa

### Creación del corpus de clasificación

En la siguiente celda, ya implementada, se crea el corpus y se subdivide en conjuntos de entrenamiento (`corpus_sentences_train`, `corpus_authors_train`), desarrollo (`corpus_sentences_dev`, `corpus_authors_dev`) y evaluación  (`corpus_sentences_test`, `corpus_authors_test`).

💡Cada conjunto está a su vez subdividido en `corpus_sentences` y `corpus_authors`, donde los textos que pertenecen a `quiroga` están señalizados por el `0`, los de `becquer` por el `1` y los de `martin_fierro` por el `2`. Esto facilita el algoritmo de partición y también la evaluación.

In [11]:
from sklearn.model_selection import train_test_split

quiroga_sentences = [x for x in nltk.sent_tokenize(quiroga,  language='spanish') if len(x)>4]
corpus_sentences = quiroga_sentences
corpus_authors = [0]*len(quiroga_sentences)

becquer_sentences = [x for x in nltk.sent_tokenize(becquer,  language='spanish') if len(x)>4]
corpus_sentences += becquer_sentences
corpus_authors += [1]*len(becquer_sentences)

martin_fierro_sentences = [x for x in nltk.sent_tokenize(martin_fierro,  language='spanish') if len(x)>4]
corpus_sentences += martin_fierro_sentences
corpus_authors += [2]*len(martin_fierro_sentences)

corpus_sentences_train, corpus_sentences_other, corpus_authors_train, corpus_authors_other = train_test_split(
    corpus_sentences, corpus_authors,
    test_size=0.3, train_size=0.7,
    random_state=2024, shuffle=True, stratify=corpus_authors)

corpus_sentences_dev, corpus_sentences_test, corpus_authors_dev, corpus_authors_test = train_test_split(
    corpus_sentences_other, corpus_authors_other,
    test_size=0.5, train_size=0.5,
    random_state=2024, shuffle=True, stratify=corpus_authors_other)

### 5️⃣ Análisis del corpus

Para entender mejor cómo está compuesto el corpus, hallen:
- la cantidad de instancias en los subconjuntos `train`, `dev` y `test`
- la cantidad de instancias de cada una de las clases (*quiroga*, *becquer*  y *martin_fierro*) en cada uno de los subconjuntos

In [20]:
from collections import Counter

#Completar

# Cantidad de instancias en cada corpus
print("---- TRAIN ----")
print(f"sentencias={len(corpus_sentences_train)}")
counter_dict = Counter(corpus_authors_train)
print(f"quiroga={counter_dict.get(0, 0)} becquer={counter_dict.get(1, 0)} martin_fierro={counter_dict.get(2, 0)}")

print("\n---- DEV ----")
print(len(corpus_sentences_dev))
counter_dict = Counter(corpus_authors_dev)
print(f"quiroga={counter_dict.get(0, 0)} becquer={counter_dict.get(1, 0)} martin_fierro={counter_dict.get(2, 0)}")

print("\n---- TEST ----")
print(len(corpus_sentences_test))
counter_dict = Counter(corpus_authors_test)
print(f"quiroga={counter_dict.get(0, 0)} becquer={counter_dict.get(1, 0)} martin_fierro={counter_dict.get(2, 0)}")


---- TRAIN ----
sentencias=4530
quiroga=2469 becquer=1727 martin_fierro=334

---- DEV ----
971
quiroga=529 becquer=370 martin_fierro=72

---- TEST ----
971
quiroga=530 becquer=370 martin_fierro=71


### Evaluación del clasificador aleatorio (línea base)

Ahora que tenemos los conjuntos definidos, ya podemos entrenar modelos. Pero antes, siempre es bueno ver cómo se comportaría un algoritmo sencillo que consista en una heurística con pocas reglas o en una clasificación aleatoria. A este modelo sencillo, que sirve como punto de comparación, le solemos llamar *línea base* (baseline).

En la siguiente celda les mostramos la evaluación sobre `dev` de la predicción aleatoria, para que tengan una línea base con la que comparar los modelos que construirán a continuación.

In [21]:
import random
from sklearn.metrics import f1_score, classification_report

random_prediction = [random.choice([0,1,2]) for i in range(len(corpus_authors_dev))]
print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, random_prediction, average='macro')*100, 2))) # Se imprime la medida F
print(classification_report(corpus_authors_dev, random_prediction, target_names=['quiroga','becquer','martin_fierro'])) # Se evalúa sobre dev

F-Score macro: 29.38
               precision    recall  f1-score   support

      quiroga       0.53      0.32      0.40       529
      becquer       0.39      0.35      0.37       370
martin_fierro       0.07      0.31      0.11        72

     accuracy                           0.33       971
    macro avg       0.33      0.33      0.29       971
 weighted avg       0.44      0.33      0.37       971



### 6️⃣ Entrenamiento y evaluación de modelos en el conjunto de desarrollo (dev)

Ahora que vimos la medida F1 en `dev` para el clasificador aleatorio, ya pueden entrenar modelos y evaluarlos intentando superar esa línea base. En la siguiente celda mostramos el entrenamiento sobre `train` de un modelo sencillo y su posterior evaluación sobre `dev`, donde usamos [bag of words](https://scikit-learn.org/stable/modules/feature_extraction.html#the-bag-of-words-representation) para representar numéricamente los textos y [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html) como algoritmo de clasificación.

Entrenen con `train` y evalúen sobre `dev` al menos 4 modelos, explorando varios algoritmos de clasificación y, si quieren, también preprocesando los textos usando  pipelines de preprocesamiento con pasos complementarios a los ya desarrollados en 4️⃣.

💡Les recomendamos que comiencen probando vectorizar con [bag of words](https://scikit-learn.org/stable/modules/feature_extraction.html#the-bag-of-words-representation) o [TF-IDF](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), y clasificando con cualquier algoritmo disponible en [sklearn](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html). Sin embargo, vemos positivo que exploren **CUALQUIER** enfoque: pueden probar usando grandes modelos de lenguaje como LLAMA o Mistral, o también vectorizando con vectores contextuales como los *sentence transformers* usados en el ejercicio 3️⃣. En [HuggingFace](https://huggingface.co/) pueden encontrar muchísimos modelos para probar.

💡Tengan en cuenta que si usan un pipeline de preprocesamiento para procesar los textos al entrenar, también tendrán que usarlo para transformar los textos al predecir.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC


bow_vectorizer = CountVectorizer(ngram_range=(2,4), strip_accents= 'unicode') # n-gramas a nivel de palabra
clf = SVC() # Prueben acá con varios modelos

training_features = bow_vectorizer.fit_transform(corpus_sentences_train) # Se vectorizan los textos de train
clf.fit(training_features, corpus_authors_train) # Se entrena el clasificador usando los textos vectorizados

dev_features = bow_vectorizer.transform(corpus_sentences_dev) # Se vectorizan los textos de dev
prediction = clf.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

F-Score macro: 33.49
               precision    recall  f1-score   support

      quiroga       0.58      1.00      0.73       529
      becquer       1.00      0.16      0.27       370
martin_fierro       0.00      0.00      0.00        72

     accuracy                           0.60       971
    macro avg       0.53      0.39      0.33       971
 weighted avg       0.70      0.60      0.50       971



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 7️⃣Evaluación final de modelos en el conjunto de evaluación (test)

Como última actividad del laboratorio, elijan los tres modelos que mejores predicciones hayan logrado sobre `dev` y evalúenlos sobre `test`.